In [2]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from qiskit_aer import QasmSimulator
from collections import defaultdict
import networkx as nx
from qiskit.circuit import ParameterVector, Parameter
from docplex.mp.model import Model
from itertools import combinations
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()
# provider = IBMProvider()

/var/folders/76/d4kyjysx1xgdrrs6zzs63s180000gn/T/ipykernel_30999/3125440094.py:12: DeprecationWarning: The "ibm_quantum" channel option is deprecated and will be sunset on 1 July. After this date, "ibm_cloud" and "local" will be the only valid channels. For information on migrating to the new IBM Quantum Platform on the "ibm_cloud" channel, review the migration guide https://quantum.cloud.ibm.com/docs/migration-guides/classic-iqp-to-cloud-iqp .
  service = QiskitRuntimeService()


In [3]:
backends = {}
backends["ibm_brisbane"] = service.backend("ibm_brisbane")

# qubits in line  experiment

In [4]:
qubits_in_line = {}
# Eagle device
qubits_1D_Eagle = list(range(13,-1,-1)) + [14] + list(range(18,33)) + [36] + list(range(51,36,-1)) + [52] + list(range(56,71)) + [74] + list(range(89,74,-1)) + [90] + list(range(94,109)) + [112] + list(range(126,112,-1))
qubits_in_line["ibm_torino"] = list(range(14,-1,-1)) + [15] + list(range(19,34)) + [37] + list(range(52,37,-1)) + [53] + list(range(57,72)) + [75] + list(range(90,75,-1)) + [91] + list(range(95,110)) + [113] + list(range(128,113,-1)) + [129]
qubits_in_line["ibm_fez"] = list(range(0,16)) + [19] + list(range(35,20,-1)) + [36] + list(range(41,56)) + [59] + list(range(75,60,-1)) + [76] + list(range(81,96)) + [99] + list(range(115,100,-1)) + [116] + list(range(121,136)) + [139] + list(range(155,139,-1))



# QAOA Circuit

In [9]:
def qaoa(gammas, betas,  G):
    p = len(gammas) 
    num_qubits = G.number_of_nodes()
    nodes = range(num_qubits)
    qc = QuantumCircuit(num_qubits, len(nodes))
    qc.h(nodes)
    for pi in range(p):
        for i, j in G.edges:
            if abs(G[i][j]["weight"]) > 1e-6:
                qc.rzz(2 * gammas[pi] * G[i][j]["weight"], i, j)
        qc.rx(-2 * betas[pi], nodes)
    qc.measure(nodes, reversed(range(len(nodes))))
    return qc

def qaoa_depth_improved(gammas, betas,  G):
    nodes = len(G.nodes())
    edges = [(i,j) for i, j in G.edges()]
    max_w = np.max(np.abs([G[i][j]["weight"] for i, j in G.edges()]))
    layers = len(gammas)
    qc = QuantumCircuit(nodes)
    qc.h(range(nodes))
    for p in range(layers):
        permutations = np.arange(nodes) # To decrease the depth of the circuit
        for jj in range(nodes):
            for k in range(jj % 2, nodes - 1, 2):
                qubit_pair = (permutations[k], permutations[k+1])
                if qubit_pair in edges or reversed(qubit_pair) in edges:
                    qc.rzz(2 * gammas[p] * G[qubit_pair[0]][qubit_pair[1]]["weight"]/max_w, *qubit_pair)
                    # qc.rzz(gammas[p], *qubit_pair)
                permutations[[k, k+1]] = permutations[[k+1, k]]
        qc.rx(-2*betas[p], range(nodes))
        # qc.rx(betas[p], range(nodes))
    # qc.measure(range(nodes), reversed(range(nodes)))
    return qc

# TKet mapping

In [5]:
from pytket.architecture import Architecture
from pytket.placement import GraphPlacement
from pytket.extensions.qiskit import qiskit_to_tk, tk_to_qiskit
from pytket.passes import RoutingPass
from pytket.transform import Transform
from pytket.extensions.qiskit.qiskit_convert import get_avg_characterisation, process_characterisation

def tket_mapping(qc, coupling_map):
    arc = Architecture(coupling_map)
    tkqc = qiskit_to_tk(qc)
    
    timeout = 30000  # Initial timeout in milliseconds (30 seconds)
    max_timeout = 3600000  # Maximum timeout set to 1 hour (3,600,000 milliseconds)

    while True:
        try:
            placer = GraphPlacement(arc, timeout=timeout)
            placer.place(tkqc)
            break  # If successful, exit the loop
        except RuntimeError as e:
            if "execution time has exceeded allowed limits" in str(e):
                if timeout >= max_timeout:
                    raise RuntimeError("GraphPlacement failed with maximum timeout")
                timeout = min(timeout * 2, max_timeout)  # Double the timeout, but cap at max_timeout
            else:
                raise  # Re-raise any other RuntimeError that isn’t timeout-related
                
    
    RoutingPass(arc).apply(tkqc)
    # Transform
    Transform.DecomposeBRIDGE().apply(tkqc)
    Transform.DecomposeSWAPtoCX(arc).apply(tkqc)
    Transform.RemoveRedundancies().apply(tkqc)
    tqc_tket_qiskit = tk_to_qiskit(tkqc)
    tqc_tket = transpile(tqc_tket_qiskit, basis_gates=["h", "rz","rx","rzz", "cx", "cz"], coupling_map=coupling_map, optimization_level=3)
    print('tket_to_qiskit_qc', tqc_tket.count_ops())
    return tqc_tket


In [6]:
from qiskit_ibm_transpiler.transpiler_service import TranspilerService
cloud_transpiler_service = TranspilerService(
    backend_name="ibm_fez",
    ai="True",
    optimization_level=3,
    use_fractional_gates=True
)

# Generate random problems with a minimum connectivity

In [7]:
seed = 3
np.random.seed(seed)
nqs = range(20, 121, 10)
problems = {"G":{}, "nqs":nqs, "probs":{}}
for nq in nqs:
    print(f"------- {nq} --------")
    min_edges = 2 * nq
    max_edges = nq * (nq - 1) / 2
    probs = np.round(np.linspace(min_edges/max_edges, 1, 10),3)
    problems["probs"][nq] = probs
    problems["G"][nq] = {}
    for prob in probs:
        num_edges = np.ceil(prob * max_edges)
        G = nx.Graph()
        G.add_nodes_from(range(nq))
        qubits = np.arange(nq)
        np.random.shuffle(qubits) # Adding a tree to ensure connectivity 
        G.add_weighted_edges_from([[qubits[i],qubits[i+1], round(np.random.rand(), 2)] for i in range(nq-1)])
        possible_edges = list(combinations(range(nq), 2))
        np.random.shuffle(possible_edges)
        for i, j in possible_edges:
            if (i,j) not in G.edges():
                G.add_weighted_edges_from([[i, j, round(np.random.rand(), 2)]])
            if G.number_of_edges() == num_edges:
                break
        problems["G"][nq][prob] = G
# np.save(f"./Data/problems_transpilers_{seed}.npy", problems)

------- 20 --------
------- 30 --------
------- 40 --------
------- 50 --------
------- 60 --------
------- 70 --------
------- 80 --------
------- 90 --------
------- 100 --------
------- 110 --------
------- 120 --------


# Running Qiskit-T, Qiskit-AI, and TKET transpilers

In [13]:
np.load("./Data/coupling_map_HE.npy", allow_pickle=True)

array([[  0,   1],
       [  1,   0],
       [  1,   2],
       [  2,   1],
       [  2,   3],
       [  3,   2],
       [  3,   4],
       [  3,  16],
       [  4,   3],
       [  4,   5],
       [  5,   4],
       [  5,   6],
       [  6,   5],
       [  6,   7],
       [  7,   6],
       [  7,   8],
       [  7,  17],
       [  8,   7],
       [  8,   9],
       [  9,   8],
       [  9,  10],
       [ 10,   9],
       [ 10,  11],
       [ 11,  10],
       [ 11,  12],
       [ 11,  18],
       [ 12,  11],
       [ 12,  13],
       [ 13,  12],
       [ 13,  14],
       [ 14,  13],
       [ 14,  15],
       [ 15,  14],
       [ 15,  19],
       [ 16,   3],
       [ 16,  23],
       [ 17,   7],
       [ 17,  27],
       [ 18,  11],
       [ 18,  31],
       [ 19,  15],
       [ 19,  35],
       [ 20,  21],
       [ 21,  20],
       [ 21,  22],
       [ 21,  36],
       [ 22,  21],
       [ 22,  23],
       [ 23,  16],
       [ 23,  22],
       [ 23,  24],
       [ 24,  23],
       [ 24,

In [16]:
from qiskit.circuit import ParameterVector
import time
methods = {"qaoa":qaoa_depth_improved}
# methods["swap_permutations"] = aoqmap_circ_noncomplete
nq = 40
nqs = [20,30,40,50,60,70,80]
nqs = [20]
seeds = [1]
methods_used = ["qaoa"]
for nq in nqs:
    print(f"/n ----- nq: {nq} -------/n")
    map_1D = [[i,i+1] for i in range(nq-1)] + [[i+1,i] for i in range(nq-1)]
    map_HE = backends["ibm_brisbane"].configuration().coupling_map
    map_HE += [[j,i] for [i,j] in map_HE]
    # map_HE = np.load("./Data/coupling_map_HE.npy", allow_pickle=True)
    gammas = ParameterVector(length=1, name=r"$\gamma$")
    betas = ParameterVector(length=1, name=r"$\beta$")

    for seed in seeds:
        print(f" ----- seed: {seed} -------")
        operations = defaultdict(dict)
        operations = np.load(f"./Data/{nq}_transpiler_{seed}.npy", allow_pickle=True).item()
        problems = np.load(f"./Data/problems_transpilers_{seed}.npy", allow_pickle=True).item()
        probs = problems["probs"][nq]
        for method_name in methods_used:
            func = methods[method_name]
            # print(f"Method: {method_name}")
            # operations["swap"] = {}
            for prob in probs:
                print(f"probs: {prob}")
                G = problems["G"][nq][prob]
                qc = func(gammas, betas, G)
                if method_name == "qaoa":
                    operations["FC"][prob] = qc.count_ops()
                    operations["FC"][prob]["depth"] = qc.depth()
                    ti = time.time()
                    qc_he = transpile(qc, coupling_map=map_HE, basis_gates=["h", "rz","rx","rzz", "cx", "cz"],optimization_level=3)
                    tf = time.time()
                    operations["HE"][prob] = qc_he.count_ops()
                    operations["HE"][prob]["time"] = tf - ti
                    operations["HE"][prob]["depth"] = qc_he.depth()

                    ti = time.time()
                    qc_1d = transpile(qc, basis_gates=["h", "rz","rx","rzz", "cx", "cz"], coupling_map=map_1D)
                    tf = time.time()
                    operations["1D"][prob] = qc_1d.count_ops()
                    operations["1D"][prob]["time"] = tf - ti
                    operations["1D"][prob]["depth"] = qc_1d.depth()
                    ti = time.time()
                    try:
                        qc_ai = cloud_transpiler_service.run(qc)
                        operations["ai"][prob] = qc_ai.count_ops()
                        operations["ai"][prob]["depth"] = qc_ai.depth()
                    except:
                        operations["ai"][prob] = {}
                        print("Couldn't find an ai solution!")
                    tf = time.time()
                    operations["ai"][prob]["time"] = tf - ti
                    ti = time.time()
                    qc_tket_he = tket_mapping(qc, coupling_map=map_HE)
                    tf = time.time()
                    operations["Tket-HE"][prob] = qc_tket_he.count_ops()
                    operations["Tket-HE"][prob]["time"] = tf - ti
                    operations["Tket-HE"][prob]["depth"] = qc_tket_he.depth()

        np.save(f"./Data/{nq}_transpiler_{seed}.npy", operations)


/n ----- nq: 20 -------/n
 ----- seed: 1 -------
probs: 0.211


ERROR:backoff:Giving up _request_transp(...) after 3 tries (requests.exceptions.HTTPError: 401 Client Error: Unauthorized for url: https://cloud-transpiler.quantum.ibm.com/transpile?backend=ibm_fez&optimization_level=3&ai=True&use_fractional_gates=True)
ERROR:qiskit_ibm_transpiler.wrappers.base:User doesn't have access to the specified backend: ibm_fez


Couldn't find an ai solution!
tket_to_qiskit_qc OrderedDict([('cx', 225), ('rzz', 41), ('h', 20), ('rx', 20)])
